In [1]:
#Residual Returns (beta adjusted returns)

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime 
from pandas_datareader import data
import statistics 

In [2]:
pd.set_option('display.precision', 4)

In [3]:
def get_start_dt(dt_, offset_days_=260):
    offset = pd.tseries.offsets.BusinessDay(n=offset_days_)
    out_dt = dt_ - offset
    return out_dt.date()

In [4]:
beta = lambda s_ret, indx_ret, range_end_dt: np.cov(s_ret.loc[:range_end_dt], indx_ret.loc[:range_end_dt])[0, 1] / \
                                             np.cov(s_ret.loc[:range_end_dt], indx_ret.loc[:range_end_dt])[1, 1]


In [5]:
res_returns = lambda tkr: ret_df[tkr].subtract(risk_free_rate) - betas[tkr] * (ret_df['SPY'].subtract(risk_free_rate))

In [6]:
def compute_residual_rets (tkr_list_):
    r = [res_returns(tkr) for tkr in tkr_list_]
    rd = pd.DataFrame(r).T
    rd.columns = tkr_list_
    return rd

In [7]:
tickers_file = 'chemicals tickers texas freeze.xlsx'
ticker = pd.read_excel(tickers_file, index_col='ticker')
ticker

,name
ticker,
YAR.OL,Yara International
MOS,Mosaic
2020.SR,SABIC Agri-Nutrients Company SJSC
CF,CF Industries Holdings Inc
DOW,Dow Chemical
LYB,LyondellBasell Industries N.V.
4092.T,Nippon Chemical
EMN,Eastman
CE,Celanese Corporation


In [8]:
tkr_list_HL = list(ticker.index)

In [9]:
#tkr_list_HL

stocks = list(ticker.reset_index().T.to_dict().values())
stocks

#stocks = [
    {
        'ticker': 'XOM',
        'name': 'Exxon'
    },
    {
        'ticker': 'CVX',
        'name': 'Chevron'
    },
    {
        'ticker': 'OXY',
        'name': 'Occidental'
    },
    {
        'ticker': 'COP',
        'name': 'ConocoPhillips'
    },
    {
        'ticker': 'EOG',
        'name': 'EOG Resources Group'
    },
    {
        'ticker': 'PXD',
        'name': 'Pioneer Natural Resources'
    },
     {
        'ticker': 'MPC',
        'name': 'Marathon Petroleum'
    },
    {
        'ticker': 'SLB',
        'name': 'Schlumberger Limited'
    },
    {
        'ticker': 'VLO',
        'name': 'Valero Energy Corporation'
    },
    {
        'ticker': 'PSX',
        'name': 'Philips 66'
    },
    {
        'ticker': 'WMB',
        'name': 'The Williams Companies Inc'
    },
    {
        'ticker': 'DVN',
        'name': 'Devon Energy'
    },
    {
        'ticker': 'KMI',
        'name': 'Kinder Morgan Inc'
    },
    {
        'ticker': 'HES',
        'name': 'HESS Corporation'
    },
    {
        'ticker': 'OKE',
        'name': 'ONEOK INC'
    },
     {
        'ticker': 'HAL',
        'name': 'Halliburton Company'
    },
    {
        'ticker': 'CTRA',
        'name': 'Coterra Energy'
    },
    {
        'ticker': 'BKR',
        'name': 'Baker Hughes Company'
    },
    {
        'ticker': 'FANG',
        'name': 'Diamondback Energy Inc'
    },
    {
        'ticker': 'MRO',
        'name': 'Marathon Oil Corporation'
    },
    {
        'ticker': 'APA',
        'name': 'APA Corporation'
    },
    {
        'ticker': 'EQT',
        'name': 'EQT Corp'
    }
]

stocks_df = pd.DataFrame(stocks)
stocks_df.set_index('ticker', inplace=True)
tkr_list = list(stocks_df.index)

tkr_list

In [10]:
crisis_dt = datetime(2021, 2, 10)
end_dt = datetime(2021, 2, 27)
start_dt = get_start_dt(end_dt, offset_days_=260)

In [11]:
cl_df = data.DataReader(['SPY'] + tkr_list_HL, start=start_dt, end=end_dt, data_source='yahoo')['Adj Close']

cl_df = data.DataReader(['SPY'] + ['EOG', 'COP'], start=start_dt, end=end_dt, data_source='yahoo')['Adj Close']

In [12]:
ret_df = cl_df.div(cl_df.shift(1)) - 1.0 
ret_df.dropna(inplace=True)
index_return = ret_df['SPY']
index_var = np.var(ret_df['SPY'])

In [13]:
betas = ret_df[tkr_list_HL].apply(beta, axis=0, indx_ret= ret_df['SPY'], range_end_dt=crisis_dt)

In [14]:
treasury_3_mo = 2.943/100
risk_free_rate = treasury_3_mo/90

In [15]:
residual_return_df = compute_residual_rets(tkr_list_HL)

In [16]:
residual_return_df

,YAR.OL,MOS,2020.SR,CF,DOW,LYB,4092.T,EMN,CE,HUN,SOLB.BR,SHEL,SSL,MEOH,DD,BAS.DE,LIN,FMC
Date,,,,,,,,,,,,,,,,,,
2020-03-03,0.0469,0.0030,0.0417,0.0314,1.6387e-02,0.0125,-0.0191,-0.0010,0.0180,-0.0141,0.0193,0.0192,0.0208,-0.0095,0.0062,0.0309,0.0145,0.0051
2020-03-04,-0.0011,-0.0504,0.0066,-0.0330,-2.1015e-02,-0.0215,0.0009,-0.0125,-0.0184,-0.0315,-0.0212,-0.0259,-0.0681,-0.0195,-0.0397,-0.0204,0.0101,0.0077
2020-03-05,-0.0004,-0.0065,-0.0044,0.0192,-7.3083e-05,0.0325,0.0141,-0.0155,0.0097,0.0296,-0.0191,0.0102,-0.0713,-0.0315,0.0016,0.0097,-0.0063,-0.0118
2020-03-10,-0.0224,-0.0111,0.0255,-0.0744,-3.8371e-02,-0.0753,-0.0002,-0.0216,0.0215,0.0309,-0.0322,-0.0037,-0.2125,-0.0539,0.0209,-0.0590,-0.0337,0.0452
2020-03-11,-0.0133,-0.0249,-0.0013,-0.0209,-4.5416e-02,-0.0228,-0.0055,-0.0299,-0.0038,-0.0096,0.0173,-0.0012,-0.1719,-0.0070,0.0218,0.0412,0.0355,0.0066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-09,-0.0069,-0.0371,0.0101,0.0013,-1.2120e-02,-0.0073,-0.0169,-0.0122,-0.0042,-0.0143,-0.0053,-0.0068,-0.0022,0.0269,-0.0293,-0.0009,-0.0060,-0.0018
2021-02-10,0.0065,-0.0025,-0.0164,-0.0165,1.9564e-02,0.0088,-0.0070,-0.0087,-0.0012,-0.0131,0.0015,0.0036,0.0018,-0.0142,-0.0423,-0.0015,-0.0109,-0.0723
2021-02-17,-0.0131,-0.0074,0.0017,-0.0042,-3.7751e-03,-0.0020,0.0105,-0.0009,0.0097,0.0026,-0.0069,0.0100,-0.0351,0.0136,-0.0143,-0.0078,-0.0037,-0.0183


In [17]:
max_by_columns = residual_return_df.max()
max_by_rows = residual_return_df.max(axis=1)

df_max = max(max_by_columns)
print("Max element based on the list of columns: ", df_max)

df_max2 = max(max_by_rows)
print("Max element based on the list of rows: ", df_max2)


Max element based on the list of columns:  0.5889891437213615
Max element based on the list of rows:  0.5889891437213615


In [18]:
median_by_columns = residual_return_df.median()
median_by_rows = residual_return_df.median(axis=1)

df_median = statistics.median(median_by_columns)
print("Median element based on the list of columns: ", df_median)

df_median2 = statistics.median(median_by_rows)
print("Median element based on the list of rows: ", df_median2)


Median element based on the list of columns:  -0.001121877501500044
Median element based on the list of rows:  -0.0015190641798934965


In [19]:
min_by_columns = residual_return_df.min()
min_by_rows = residual_return_df.min(axis=1)

df_min = min(min_by_columns)
print("Minimum element based on the list of columns: ", df_min)

df_min2 = min(min_by_rows)
print("Minimum  element based on the list of rows: ", df_min2)


Minimum element based on the list of columns:  -0.25426010685611483
Minimum  element based on the list of rows:  -0.25426010685611483
